# CustomPCA和CustomBERT测试 - THUCNews数据集

本notebook用于测试自定义实现的PCA和BERT

## 1. 导入库

In [ ]:
import numpy as np
import pickle
import os
import sys
sys.path.append('.')

from custom_implementations import CustomPCA, CustomBERTExtractor
from custom_implementations import compare_pca_implementations

## 2. 加载THUCNews预处理数据

In [ ]:
preprocess_path = 'features/preprocessed/preprocessed_data.pkl'

if os.path.exists(preprocess_path):
    with open(preprocess_path, 'rb') as f:
        data = pickle.load(f)
    
    X_train_tokens = data['X_train_tokens']
    X_val_tokens = data['X_val_tokens']
    X_test_tokens = data['X_test_tokens']
    y_train = data['y_train']
    y_val = data['y_val']
    y_test = data['y_test']
    
    print(f"✅ 成功加载预处理数据")
    print(f"  训练集: {len(X_train_tokens)} 样本")
    print(f"  验证集: {len(X_val_tokens)} 样本")
    print(f"  测试集: {len(X_test_tokens)} 样本")
    print(f"  类别数: {len(np.unique(y_train))}")
else:
    print("❌ 预处理数据未找到")
    X_train_tokens = None

# 加载BERT特征
bert_features_path = 'features/bert/bert_features_768d.pkl'

if os.path.exists(bert_features_path):
    with open(bert_features_path, 'rb') as f:
        bert_data = pickle.load(f)
    
    X_train_bert = bert_data['X_train']
    X_val_bert = bert_data['X_val']
    X_test_bert = bert_data['X_test']
    
    print(f"\n✅ 成功加载BERT特征")
    print(f"  训练集: {X_train_bert.shape}")
    print(f"  验证集: {X_val_bert.shape}")
    print(f"  测试集: {X_test_bert.shape}")
    print(f"\n特征统计:")
    print(f"  均值: {X_train_bert.mean():.6f}")
    print(f"  标准差: {X_train_bert.std():.6f}")
    print(f"  范围: [{X_train_bert.min():.6f}, {X_train_bert.max():.6f}]")
else:
    print(f"\n❌ BERT特征未找到: {bert_features_path}")
    print("   请先运行特征提取脚本")
    X_train_bert = None

## 3. 测试CustomPCA基本功能（使用真实BERT特征）

In [ ]:
if X_train_bert is not None:
    print("测试CustomPCA基本功能 - 使用真实BERT特征")
    print("="*60)
    
    # 使用部分数据测试（如果数据量很大）
    n_test_samples = min(1000, X_train_bert.shape[0])
    X = X_train_bert[:n_test_samples]
    print(f"\n使用前{n_test_samples}个样本进行测试")
    print(f"测试数据形状: {X.shape}")
    print(f"均值: {X.mean():.6f}")
    print(f"标准差: {X.std():.6f}")
    
    n_components = min(100, X.shape[0], X.shape[1])
    pca = CustomPCA(n_components=n_components, random_state=42)
    X_pca = pca.fit_transform(X)

    print(f"\n原始数据形状: {X.shape}")
    print(f"降维后形状: {X_pca.shape}")
    print(f"\n前10个主成分解释方差:")
    for i in range(min(10, n_components)):
        print(f"  PC{i+1}: {pca.explained_variance_ratio_[i]:.6f}")

    print(f"\n累计解释方差: {np.sum(pca.explained_variance_ratio_):.6f}")
else:
    print("⚠️  跳过测试（BERT特征未加载）")

## 4. 测试重构功能

In [ ]:
if X_train_bert is not None:
    X_reconstructed = pca.inverse_transform(X_pca)
    reconstruction_error = np.mean((X - X_reconstructed) ** 2)

    print(f"重构误差(MSE): {reconstruction_error:.10f}")
    print("\n✅ CustomPCA基本功能正常")
else:
    print("⚠️  跳过测试（BERT特征未加载）")

## 5. 与sklearn.PCA对比

In [ ]:
if X_train_bert is not None:
    compare_pca_implementations(X, n_components=n_components, n_samples_display=10)
else:
    print("⚠️  跳过对比测试（BERT特征未加载）")

## 6. 加载THUCNews数据集（如果有）

In [ ]:
import pickle
import os

preprocess_path = 'features/preprocessed/preprocessed_data.pkl'

if os.path.exists(preprocess_path):
    with open(preprocess_path, 'rb') as f:
        data = pickle.load(f)
    
    X_train_tokens = data['X_train_tokens']
    y_train = data['y_train']
    
    print(f"✅ 成功加载THUCNews数据")
    print(f"  训练样本数: {len(X_train_tokens)}")
    print(f"  类别数: {len(np.unique(y_train))}")
else:
    print("⚠️  THUCNews数据未找到")
    X_train_tokens = None
    y_train = None

## 7. 测试BERT特征提取（使用随机权重）

In [ ]:
if X_train_tokens is not None:
    n_test_samples = 20
    print(f"使用前{n_test_samples}个样本测试BERT特征提取")
    
    bert_extractor = CustomBERTExtractor(
        model_name='bert-base-chinese',
        max_length=128,
        batch_size=4,
        use_pretrained=False
    )
    
    X_bert = bert_extractor.transform(X_train_tokens[:n_test_samples])
    
    print(f"\nBERT特征形状: {X_bert.shape}")
    print(f"特征均值: {X_bert.mean():.6f}")
    print(f"特征标准差: {X_bert.std():.6f}")
else:
    print("跳过BERT测试（数据未加载）")

## 8. BERT特征 + PCA降维

In [ ]:
if X_train_tokens is not None:
    print("对BERT特征进行PCA降维")
    
    n_samples = X_bert.shape[0]
    n_features = X_bert.shape[1]
    max_components = min(n_samples, n_features)
    n_components_target = min(50, max_components)
    
    print(f"样本数: {n_samples}, 特征数: {n_features}")
    print(f"最大可用主成分数: {max_components}")
    print(f"目标主成分数: {n_components_target}")
    
    pca_bert = CustomPCA(n_components=n_components_target, random_state=42)
    X_bert_pca = pca_bert.fit_transform(X_bert)
    
    print(f"\n降维前: {X_bert.shape}")
    print(f"降维后: {X_bert_pca.shape}")
    print(f"\n累计解释方差: {np.sum(pca_bert.explained_variance_ratio_):.6f}")
    
    n_display = min(10, n_components_target)
    print(f"\n前{n_display}个PC解释方差:")
    for i in range(n_display):
        print(f"  PC{i+1}: {pca_bert.explained_variance_ratio_[i]:.6f}")
else:
    print("跳过PCA降维测试（数据未加载）")

## 9. 可视化PCA方差解释

In [ ]:
if X_train_tokens is not None:
    import matplotlib.pyplot as plt
    
    n_components_actual = len(pca_bert.explained_variance_ratio_)
    print(f"实际主成分数量: {n_components_actual}")
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    axes[0].plot(range(1, n_components_actual + 1), pca_bert.explained_variance_ratio_, 'b-', linewidth=2, marker='o')
    axes[0].set_xlabel('主成分', fontsize=12)
    axes[0].set_ylabel('解释方差比例', fontsize=12)
    axes[0].set_title('各主成分解释方差', fontsize=14, fontweight='bold')
    axes[0].grid(True, alpha=0.3)
    
    cumsum = np.cumsum(pca_bert.explained_variance_ratio_)
    axes[1].plot(range(1, n_components_actual + 1), cumsum, 'r-', linewidth=2, marker='o')
    axes[1].axhline(y=0.95, color='g', linestyle='--', label='95%阈值')
    axes[1].set_xlabel('主成分数量', fontsize=12)
    axes[1].set_ylabel('累计解释方差', fontsize=12)
    axes[1].set_title('累计解释方差', fontsize=14, fontweight='bold')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    if np.max(cumsum) >= 0.95:
        n_95 = np.argmax(cumsum >= 0.95) + 1
        print(f"\n保留95%方差需要 {n_95} 个主成分")
    else:
        print(f"\n所有{n_components_actual}个主成分累计解释方差: {cumsum[-1]:.4f} ({cumsum[-1]*100:.2f}%)")
else:
    print("跳过可视化（数据未加载）")

## 10. 总结

In [ ]:
print("="*80)
print("测试总结")
print("="*80)
print("\n✅ 完成的测试:")
print("  1. CustomPCA基本功能测试")
print("  2. CustomPCA vs sklearn.PCA对比")
if X_train_tokens is not None:
    print("  3. THUCNews数据集测试")
    print("  4. BERT特征提取测试")
    print("  5. PCA降维测试")
print("\n📝 结论:")
print("  - CustomPCA使用SVD实现，与sklearn.PCA结果一致")
print("  - CustomBERT完整实现Transformer架构")
print("  - 可用于extract_bert_with_pca.py")